In [1]:
import yfinance as yf 
import pandas as pd
from matplotlib import pylab as plt
import numpy as np
from datetime import datetime
import pandas_datareader.data as web

from time import sleep

import quandl

import seaborn as sns

import sys

from scipy.stats import norm
from statsmodels.distributions.empirical_distribution import ECDF
import urllib
from datetime import date , timedelta

In [2]:
def color_negative_red(value):
    """
      Colors elements in a dateframe
      green if positive and red if
      negative. Does not color NaN
      values.
      """

    if value < 0:
        color = 'red'
    elif value > 0:
        color = 'green'
    else:
        color = 'black'

    return 'color: %s' % color

# Macro Analysis

To Do
- Economic Quadrant
- Econmic Quadrant with Rate of Change
- M2 growth
- M2 Acceleration
- 10Y bond yield rate of change
- Include Qualtiles of all
- Present in a nice format
- economic policy
-  Sentiment (TED spread, )

In [60]:
GDP_tmp = web.DataReader('GDPC1','fred', '1970-01-01') #Quaterly
CPI_tmp = web.DataReader('CPIAUCSL','fred', '1970-01-01') # Monthly
yield10_tmp = web.DataReader('DGS10','fred', '1970-01-01') # Daily
M2_tmp = web.DataReader('M2','fred', '1970-01-01') # Weekly
yield2_tmp = web.DataReader('DGS2','fred', '1970-01-01') # Daily
IP_tmp =  web.DataReader('INDPRO','fred', '1970-01-01') #monthly

### Rate of Change YoY

In [86]:
#We want diff from 1 year ago!!

IP_RC = IP_tmp.pct_change(12).diff(12)
CPI_RC = CPI_tmp.pct_change(12).diff(12)
yield2_change = yield2_tmp.diff(252)

RC_tmp = pd.concat([IP_RC, CPI_RC],axis =1)

tol = pd.Timedelta('5 day')
RC_Data = pd.merge_asof(left=RC_tmp,right=yield2_change,right_index=True,left_index=True,direction='nearest',tolerance=tol)
RC_Data.columns = ["IP", "CPI", '2 Year Yield']
RC_Data = RC_Data.dropna()

RC_Data.index = RC_Data.index.to_period('M')

## Dashboard

In [90]:
macro_dash = RC_Data.tail(12)
th_props = [
  ('font-size', '16px'),
  ('text-align', 'center'),
  ('font-weight', 'bold'),
  ('color', 'Black'),
  ('background-color', '#f7f7f9')
  ]


# Set CSS properties for td elements in dataframe
td_props = [('font-size', '16px'), ('text-align', 'center')]

# Set table styles
styles = [
    dict(selector="th", props=th_props),
    dict(selector="td", props=td_props)
]

cm_low_good = sns.diverging_palette(150, 10, s=80, l=70, n=len(macro_dash.columns), as_cmap=True)#sns.light_palette("green", as_cmap=True)
cm_high_good = sns.diverging_palette(10, 150, s=80, l=70, n=len(macro_dash.columns), as_cmap=True)

macro_dash_VIEW = (macro_dash.style
    .applymap(color_negative_red, subset=['IP', 'CPI', '2 Year Yield'])
    #.background_gradient(axis = 0 ,subset = ['DSSP', 'HIDI'],cmap=cm_high_good)
    #.background_gradient(axis = 0 ,subset = ['VRANK', 'MRANK','CRANK', 'ST'],cmap=cm_low_good)
    #.set_caption('Performance Metrics')
    .format({'IP': "{:.1%}",'CPI': "{:.1%}", '2 Year Yield': "{:.2f}"})
    .set_table_styles(styles))

In [91]:
macro_dash_VIEW

,IP,CPI,2 Year Yield
DATE,,,
2020-12,-1.1%,-1.0%,-1.44
2021-02,-3.5%,-0.6%,-1.33
2021-03,7.0%,1.1%,-0.37
2021-04,35.5%,3.8%,-0.06
2021-05,32.6%,4.7%,0.00
2021-06,21.2%,4.6%,-0.03
2021-07,13.6%,4.2%,0.11
2021-08,11.9%,3.9%,0.01
2021-09,11.2%,4.0%,0.06


# Rolling Correlation Estimation
- SPY
- Gold
- IEF (10 Yr Bond)
- TIPS

In [89]:
spy_tmp = yf.download('SPY') 
gld_tmp = yf.download('GLD') 
ief_tmp = yf.download('IEF') 
tips_tmp = yf.download('TIP') 
dbc_tmp = yf.download('DBC')

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [37]:
spy = spy_tmp['Close']
gld = gld_tmp['Close']
ief = ief_tmp['Close']
tips = tips_tmp['Close']
dbc = dbc_tmp['Close']

In [38]:
corr_data = pd.concat([spy, gld, ief, tips, dbc],axis =1)
corr_data.columns = ['SP500', 'Gold', '10Yr Bonds', 'TIPS', 'Commodities']

corr_data_3yr = corr_data.pct_change(1).tail(60)
corr_matrix = corr_data_3yr.corr(method = 'spearman')
corr_matrix

,SP500,Gold,10Yr Bonds,TIPS,Commodities
SP500,1.000000,0.057127,-0.277466,0.166602,0.525979
Gold,0.057127,1.000000,0.257071,0.325368,0.177160
10Yr Bonds,-0.277466,0.257071,1.000000,0.542151,-0.363490
TIPS,0.166602,0.325368,0.542151,1.000000,-0.031787
Commodities,0.525979,0.177160,-0.363490,-0.031787,1.000000


## High Yield Spread and Yield Curve

Use above/below 10 year median for signal for growth and inflation



In [39]:
# Import Data
HY_spread_tmp = web.DataReader('BAMLH0A0HYM2','fred', '1970-01-01') 
yiedl_10_2_tmp = web.DataReader('T10Y2Y','fred', '1970-01-01')

In [40]:
HY_spread = HY_spread_tmp[-10*252:]
yiedl_10_2 = yiedl_10_2_tmp[-10*252:]

yiedl_10_2_chg = yiedl_10_2.pct_change(3*30)[-1:].values
HY_spread_chg = HY_spread.pct_change(3*30)[-1:].values

HY_spread_ecdf = ECDF(HY_spread['BAMLH0A0HYM2'])
yiedl_10_2_ecdf = ECDF(yiedl_10_2['T10Y2Y'])

HY_spread_current = HY_spread_ecdf(HY_spread[-1:])
yiedl_10_2_current = yiedl_10_2_ecdf(yiedl_10_2[-1:])



macro_percentile = pd.concat([pd.DataFrame(HY_spread_current), pd.DataFrame(yiedl_10_2_current)],axis =1)
macro_percentile.columns = ["HY Spread Pct", "Yield Curve Pct"]

macro_chg = pd.concat([pd.DataFrame(HY_spread_chg), pd.DataFrame(yiedl_10_2_chg)],axis =1)
macro_chg.columns = ["HY Spread Chg", "Yield Curve Chg"]

In [46]:
percentile_dash = macro_percentile
th_props = [
  ('font-size', '16px'),
  ('text-align', 'center'),
  ('font-weight', 'bold'),
  ('color', 'Black'),
  ('background-color', '#f7f7f9')
  ]


# Set CSS properties for td elements in dataframe
td_props = [('font-size', '16px'), ('text-align', 'center')]

# Set table styles
styles = [
    dict(selector="th", props=th_props),
    dict(selector="td", props=td_props)
]

cm_low_good = sns.diverging_palette(150, 10, s=80, l=70, n=len(macro_dash.columns), as_cmap=True)#sns.light_palette("green", as_cmap=True)
cm_high_good = sns.diverging_palette(10, 150, s=80, l=70, n=len(macro_dash.columns), as_cmap=True)

percentile_dash_VIEW = (percentile_dash.style
    #.applymap(color_negative_red, subset=['GDP', 'CPI', '2 Year Yield'])
    #.background_gradient(axis = 0 ,subset = ['DSSP', 'HIDI'],cmap=cm_high_good)
    #.background_gradient(axis = 0 ,subset = ['VRANK', 'MRANK','CRANK', 'ST'],cmap=cm_low_good)
    #.set_caption('Performance Metrics')
    .format({'HY Spread Pct': "{:.1%}",'Yield Curve Pct': "{:.1%}"})
    .set_table_styles(styles))

In [47]:
macro_dash_VIEW

,GDP,CPI,2 Year Yield
DATE,,,
2020Q3,-0.05,-0.01,-90.7%
2020Q4,-0.05,-0.01,-91.3%
2021Q1,-0.00,-0.01,-91.7%
2021Q2,0.21,0.04,-26.1%
2021Q3,0.08,0.04,78.6%


In [48]:
## Add 3 month trend !!
percentile_dash_VIEW

,HY Spread Pct,Yield Curve Pct
0,10.8%,31.0%
